# Scrape Nigerian Election Results

This is the first notebook used to scrape the Nigerian presidential 2023 election results from Lagos. This notebook collects the links to the wards in each LGA.

In [ ]:
import requests as req
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import json
import urllib
import os
import logging


import pandas as pd
# create dictionary of all LGAs : Wards

base_url = 'https://inecelectionresults.ng/elections/63f8f25b594e164f8146a213?state=25'
homepage = 'https://inecelectionresults.ng'
LGA_dict = {}
headers = req.utils.default_headers()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
source = req.get(base_url, headers=headers).text
soup = BeautifulSoup(source, "html.parser")
driver = webdriver.Chrome(ChromeDriverManager().install())

# open base url with selenium
driver.get(base_url)

# wait for page to load
time.sleep(2)

soup = BeautifulSoup(driver.page_source, "html.parser")
# find all LGA names: all elements 'a' with class = bold
LGAs = soup.find_all('a', class_ = 'bold')
# extract text from each element
LGA_names = [LGA.text for LGA in LGAs]
# extract href from each element
LGA_links = [homepage + LGA['href'] for LGA in LGAs]
# open each LGA page and extract all ward names and links
for i in range(len(LGA_links)):
    LGA_dict[LGA_names[i]] = {}
    # open LGA page with selenium
    driver.get(LGA_links[i])
    # wait for page to load
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # find all ward names: all elements 'a' with class = bold
    wards = soup.find_all('a', class_ = 'bold')
    # extract text from each element
    ward_names = [ward.text for ward in wards]
    # extract href from each element
    ward_links = [ward['href'] for ward in wards]
    # add ward names and links to dictionary
    for j in range(len(ward_links)):
        LGA_dict[LGA_names[i]][ward_names[j]] = ward_links[j]
# create new dictionary with layout LGA : ward : ward_link = link
LGA_dict2 = {}
for LGA in LGA_dict:
    LGA_dict2[LGA] = {}
    for ward in LGA_dict[LGA]:
        LGA_dict2[LGA][ward] = {} 
        LGA_dict2[LGA][ward] = {'link':homepage + LGA_dict[LGA][ward]}
# add value 'done' to each ward in dictionary
for LGA in LGA_dict2:
    for ward in LGA_dict2[LGA]:
        # add key value pair 'done' to each ward in dictionary
        LGA_dict2[LGA][ward]['done'] = False

# save dictionary to json file
import json
with open('LGA_dict.json', 'w') as fp:
    json.dump(LGA_dict2, fp)
    